<a href="https://colab.research.google.com/github/nataliaespector/CEIA-VisionPorComputadoraII/blob/main/2_Eliminar_duplicados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Final Visión por Computadora II - CEIA - UBA
## Eliminar imágenes duplicadas
### Dataset seleccionado: Chest CT-Scan images Dataset (Kaggle).

Objetivo: Luego del EDA inicial, se observó que el dataset de 1000 imágenes contaba con 153 duplicados exactos. Al eliminarlos, se observó que algunos de los split quedaban con muy pocas imágenes. Por esta razón, se decidió unir el dataset, eliminar duplicados y volver a realizar el split en train, test y valid. Se genera un nuevo dataset *Data_Clean* que se utilizará para los entrenamientos posteriores.

## Importar librerías

In [1]:
!pip install imagehash

In [2]:
import os
import hashlib
import random
from pathlib import Path
from collections import defaultdict
import shutil

from PIL import Image
import imagehash
import cv2

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Carga de datos


Leer carpeta cargada en Drive y verificar su contenido

In [3]:
data_dir = '/content/drive/MyDrive/Data'

if os.path.exists(data_dir):
    print(f"Contenido de {data_dir}:")
    for item in os.listdir(data_dir):
        print(item)
else:
    print(f"El directorio {data_dir} no existe.")

Contenido de /content/drive/MyDrive/Data:
train
test
valid


Verificar los subdirectorios `train`, `test`, y `valid` y su contenido



In [4]:
for subdir in ['train', 'test', 'valid']:
    subdir_path = os.path.join(data_dir, subdir)
    if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
        print(f"\nContenido de {subdir_path}:")
        contents = os.listdir(subdir_path)
        for i, item in enumerate(contents):
            print(f"  {item}")
    else:
        print(f"\nEl directorio {subdir_path} no existe.")


Contenido de /content/drive/MyDrive/Data/train:
  adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib
  large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa
  squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa
  normal

Contenido de /content/drive/MyDrive/Data/test:
  large.cell.carcinoma
  normal
  adenocarcinoma
  squamous.cell.carcinoma

Contenido de /content/drive/MyDrive/Data/valid:
  adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib
  large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa
  squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa
  normal


## Eliminar imágenes duplicadas

Se utilizan funciones previamente utilizadas para el EDA para buscar las imágenes duplicadas mediante el hash MD5.

In [5]:
# Directorio raíz del dataset original
data_dir = Path(data_dir)

# Extensiones de imagen válidas
image_extensions = {'.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'}

# Función para normalizar nombres de clases
def normalize_class_name(class_name):
    if class_name == 'normal':
        return 'normal'
    elif class_name.startswith('adenocarcinoma'):
        return 'adenocarcinoma_left'
    elif class_name.startswith('large.cell.carcinoma'):
        return 'large.cell.carcinoma_left'
    elif class_name.startswith('squamous.cell.carcinoma'):
        return 'squamous.cell.carcinoma_left'
    else:
        return class_name

# Función para calcular hash perceptual (dhash)
def calculate_dhash(image_path):
    """Calcula el hash perceptual (dhash) de una imagen"""
    try:
        img = Image.open(image_path)
        # Convertir a RGB si es necesario
        if img.mode != 'RGB':
            img = img.convert('RGB')
        # Redimensionar a 9x8 para dhash
        img = img.resize((9, 8), Image.Resampling.LANCZOS)
        # Convertir a escala de grises
        img = img.convert('L')
        # Calcular dhash
        hash_value = imagehash.dhash(img)
        return str(hash_value)
    except Exception as e:
        return None

# Función para calcular hash MD5 (duplicados exactos)
def calculate_md5(image_path):
    """Calcula el hash MD5 de un archivo"""
    try:
        with open(image_path, 'rb') as f:
            return hashlib.md5(f.read()).hexdigest()
    except Exception as e:
        return None

print("Buscando imágenes y calculando hashes...")
print("=" * 80)

image_info = []

for split in ['train', 'valid', 'test']:
    split_path = data_dir / split
    if not split_path.exists():
        continue

    for class_folder in split_path.iterdir():
        if not class_folder.is_dir():
            continue

        class_name = class_folder.name
        normalized_class = normalize_class_name(class_name)

        for image_file in class_folder.iterdir():
            if image_file.is_file() and image_file.suffix in image_extensions:
                md5 = calculate_md5(image_file)
                dhash = calculate_dhash(image_file)

                info = {
                    'Split': split,
                    'Clase': normalized_class,
                    'Clase_original': class_name,
                    'Archivo': image_file.name,
                    'Ruta': str(image_file),
                    'MD5': md5,
                    'DHash': dhash,
                }
                image_info.append(info)

# Pasar a DataFrame
df = pd.DataFrame(image_info)
print("Total de imágenes encontradas:", len(df))
print(df.head())

Buscando imágenes y calculando hashes...
Total de imágenes encontradas: 1000
   Split                Clase                              Clase_original  \
0  train  adenocarcinoma_left  adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib   
1  train  adenocarcinoma_left  adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib   
2  train  adenocarcinoma_left  adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib   
3  train  adenocarcinoma_left  adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib   
4  train  adenocarcinoma_left  adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib   

          Archivo                                               Ruta  \
0  000015 (9).png  /content/drive/MyDrive/Data/train/adenocarcino...   
1  000009 (7).png  /content/drive/MyDrive/Data/train/adenocarcino...   
2  000021 (5).png  /content/drive/MyDrive/Data/train/adenocarcino...   
3  000021 (4).png  /content/drive/MyDrive/Data/train/adenocarcino...   
4  000015 (4).png  /content/drive/MyDrive/Data/train/adenocarcino...   

                           

In [6]:
# Eliminar filas sin MD5 (si hubo errores de lectura)
df_clean = df.dropna(subset=['MD5']).copy()

# Cantidad por clase antes de deduplicar
print("\nImágenes por clase ANTES de deduplicar:")
print(df_clean['Clase'].value_counts())

# Mantener una sola imagen por MD5 (la primera)
df_dedup = df_clean.sort_values('Ruta').drop_duplicates(subset=['MD5'], keep='first')

print("\nImágenes por clase DESPUÉS de deduplicar (MD5):")
print(df_dedup['Clase'].value_counts())

print("\nTotal antes:", len(df_clean), " | Total después de deduplicar:", len(df_dedup))



Imágenes por clase ANTES de deduplicar:
Clase
adenocarcinoma_left             338
squamous.cell.carcinoma_left    260
normal                          215
large.cell.carcinoma_left       187
Name: count, dtype: int64

Imágenes por clase DESPUÉS de deduplicar (MD5):
Clase
adenocarcinoma_left             337
squamous.cell.carcinoma_left    257
large.cell.carcinoma_left       187
normal                           66
Name: count, dtype: int64

Total antes: 1000  | Total después de deduplicar: 847


## Nuevo split

Se realiza nuevamente la separación entre train, test y valid con el df sin duplicados

In [7]:
test_size = 0.15
valid_size = 0.15
random_state = 42

# Separar test
df_rest, df_test = train_test_split(
    df_dedup,
    test_size=test_size,
    stratify=df_dedup['Clase'],
    random_state=random_state
)

# Separar train y valid
valid_ratio = valid_size / (1 - test_size)

df_train, df_valid = train_test_split(
    df_rest,
    test_size=valid_ratio,
    stratify=df_rest['Clase'],
    random_state=random_state
)

print("\nTamaños finales:")
print("Train:", len(df_train))
print("Valid:", len(df_valid))
print("Test: ", len(df_test))

print("\nClases en TRAIN:")
print(df_train['Clase'].value_counts())

print("\nClases en VALID:")
print(df_valid['Clase'].value_counts())

print("\nClases en TEST:")
print(df_test['Clase'].value_counts())


Tamaños finales:
Train: 592
Valid: 127
Test:  128

Clases en TRAIN:
Clase
adenocarcinoma_left             235
squamous.cell.carcinoma_left    180
large.cell.carcinoma_left       131
normal                           46
Name: count, dtype: int64

Clases en VALID:
Clase
adenocarcinoma_left             51
squamous.cell.carcinoma_left    38
large.cell.carcinoma_left       28
normal                          10
Name: count, dtype: int64

Clases en TEST:
Clase
adenocarcinoma_left             51
squamous.cell.carcinoma_left    39
large.cell.carcinoma_left       28
normal                          10
Name: count, dtype: int64


## Generar nueva carpeta Data_Clean

Se crea el nuevo directorio con los datos sin duplicados y separados según el nuevo split

In [8]:
# Nuevo directorio Data_Clean
new_root = data_dir.parent / "Data_Clean"

def copiar_split(df_split, split_name, root_dest):
    for _, row in df_split.iterrows():
        src = Path(row['Ruta'])
        clase = row['Clase']
        dest_dir = root_dest / split_name / clase
        dest_dir.mkdir(parents=True, exist_ok=True)
        dest = dest_dir / src.name

        if dest.exists():
            continue

        shutil.copy2(src, dest)

    print(f"Copiadas {len(df_split)} imágenes a {split_name}/")

# Crear nueva estructura limpia
copiar_split(df_train, 'train', new_root)
copiar_split(df_valid, 'valid', new_root)
copiar_split(df_test,  'test',  new_root)

Copiadas 592 imágenes a train/
Copiadas 127 imágenes a valid/
Copiadas 128 imágenes a test/
